In [ ]:
import pandas as pd
import yaml
import unicodedata
import numpy as np
import re
import Levenshtein
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
from jiwer import wer, cer
from typing import Set
import pandas as pd
import numpy as np
import Levenshtein

warnings.filterwarnings('ignore')

# 1. Carregando os Dados


In [ ]:
confs = yaml.safe_load(open("confs.yaml"))
predictors = confs["predictors"] ### Importante! O cientista poderá usar apenas estas features para criar/aperfeiçoar o modelo
text_target = confs["text_target"]
cols_to_keep = predictors + text_target + ['cnpj']
df = pd.read_parquet("dados/train.parquet")[cols_to_keep]

# 2. Preparando os Dados

## 2.1 Presenca de Valores `NaN`

In [ ]:
print("\nValores NaN por coluna:")
print(df.isnull().sum())
print(f"\nTotal de linhas com algum valor NaN: {df.isnull().any(axis=1).sum()}")

Como nenhuma das colunas do conjunto de dados fornecido possui algum valor `NaN`, nao será necessária a realizacao de nenhum tratamento desse tipo.

## 2.2 Presenca de Linhas Duplicadas

Linhas duplicadas em relacao a todas as colunas do conjunto de dados serao removidas.

In [ ]:
duplicate_count = df.duplicated().sum()
print(f"Número de linhas duplicadas: {duplicate_count}")

In [ ]:
df_cleaned = df.drop_duplicates()

print(f"Tamanho do conjunto original: {df.shape}")
print(f"Tamanho do conjunto após remocao de linhas duplicadas em relacao a todas as colunas: {df_cleaned.shape}")

## 2.3 Remocao de Termos Irrelevantes

Colunas com o sufixo `_cleaned` serao criadas para as colunas `user_input`, `razaosocial` e `nome_fantasia`. Nelas, termos como "S.A.", "LTDA", "LTDA.", "S/A", "S.A", "Ltda", "Ltda.", "S/A.", "S.A.", "S.A", "Ltda" e "Ltda" serao removidos baseados na seguinte suposicao:

 > **Usuários nao tem o hábito de utilizar esses termos ao se referir a nomes de empresas, então elas não devem ser consideradas na busca/retrieval**


Essa remocao é importante, pois ao se calcular a similaridade entre um `user_input` e/ou `razaosocial` e `nome_fantasia`, as métricas de similaridade seriam prejudicadas na ausencia de tais termos no `user_input`. 

Também serao removidas acentos e stopwords da língua portuguesa, sobretudo preposicoes. Tais termos, por nao informarem sobre empresas específicas, podem prejudicar a acurácia das buscas/retrieval.
Além disso, passaremos tudo para letras minúsculas, para evitar problemas de case-sensitive.

In [ ]:
def comprehensive_text_cleaning(text, 
                               remove_accents=True,
                               remove_stop_words=True, 
                               remove_company_suffixes=True,
                               custom_stop_words=None,
                               to_lowercase=True):
    """
    Comprehensive text cleaning function
    
    Parameters:
    text (str): Input text
    remove_accents (bool): Remove accents and normalize characters
    remove_stop_words (bool): Remove Portuguese stop words
    remove_company_suffixes (bool): Remove common company suffixes
    custom_stop_words (set): Additional stop words to remove
    to_lowercase (bool): Convert to lowercase
    
    Returns:
    str: Cleaned text
    """
    
    if pd.isna(text):
        return text
    
    text = str(text)
    
    if remove_accents:
        text = unicodedata.normalize('NFD', text)
        text = ''.join(char for char in text if unicodedata.category(char) != 'Mn')
        text = text.replace('ç', 'c').replace('Ç', 'C')
    
    if to_lowercase:
        text = text.lower()
    
    if remove_company_suffixes:
        patterns_to_remove = [
        r'\bS\.?A\.?\b',           # S.A, SA, S.A., SA.
        r'\bS/A\.?\b',             # S/A, S/A.
        r'\bLTDA\.?\b',            # LTDA, LTDA.
        r'\bLIMITADA\b',           # LIMITADA
        r'\bCIA\.?\b',             # CIA, CIA.
        r'\bCOMPANHIA\b',          # COMPANHIA
        r'\bEMPRESA\b',            # EMPRESA
        r'\bCOMERCIO\b',           # COMERCIO
        r'\bSERVICOS?\b',          # SERVICO, SERVICOS
        r'\bME\b',                 # ME (Microempresa)
        r'\bEPP\b',                # EPP (Empresa de Pequeno Porte)
        r'\bEIRELI\b',             # EIRELI
        r'\bSOCIEDADE\b',          # SOCIEDADE
        r'ADMINISTRADORA\b',       # ADMINISTRADORA
        r'GERAL\b',                # GERAL
    ]
        
        for pattern in patterns_to_remove:
            text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    
    if remove_stop_words:
        portuguese_stop_words = {
            'a', 'ao', 'aos', 'as', 'da', 'das', 'de', 'do', 'dos', 'e', 'em', 'na', 
            'nas', 'no', 'nos', 'o', 'os', 'para', 'por', 'com', 'um', 'uma', 'uns', 
            'umas', 'se', 'que', 'ou', 'mas', 'como', 'mais', 'muito', 'sua', 'seu',
            'seus', 'suas', 'este', 'esta', 'estes', 'estas', 'esse', 'essa', 'esses',
            'essas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'isso', 'aquilo'
        }
        
        if custom_stop_words:
            portuguese_stop_words.update(custom_stop_words)
        
        words = text.split()
        words = [word for word in words if word.lower() not in portuguese_stop_words]
        text = ' '.join(words)
    
    text = re.sub(r'[^\w\s]', ' ', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text)      # Multiple spaces to single space
    text = text.strip()                   # Remove leading/trailing spaces
    
    return text

In [ ]:
df_cleaned['user_input_cleaned'] = df['user_input'].apply(comprehensive_text_cleaning)
df_cleaned['razaosocial_cleaned'] = df['razaosocial'].apply(comprehensive_text_cleaning)
df_cleaned['nome_fantasia_cleaned'] = df['nome_fantasia'].apply(comprehensive_text_cleaning)

# 3. Retriever com Métodos Clássicos 

## 3.1 Definicao das Métricas

As seguintes métricas de similaridade e erro entre strings a nível de caracter serao implementadas:

***
> ## Word Error Rate (WER): 

 Calcula a taxa de erro a nível de palavras: 
  $$WER = \frac{S + D + I}{N}$$
  onde:
  - $S$ é o número de substituições. Por exemplo, se o usuário digitou "Empresa X" e a referência é "Empresa Y", então há uma substituição.
  - $D$ é o número de deleções. Por exemplo, se o usuário digitou "Empresa" e a referência é "Empresa X", então há uma deleção.
  - $I$ é o número de inserções. Por exemplo, se o usuário digitou "Empresa X Y" e a referência é "Empresa X", então há uma inserção.
  - $N$ é o número total de palavras na referência. Por exemplo, se a referência é "Empresa X", então $N$ é 2.

***
> ## Character Error Rate (CER): 

Calcula a taxa de erro a nível de caracteres:
  $$CER = \frac{S + D + I}{N}$$
  onde:
  - $S$ é o número de substituições. Por exemplo, se o usuário digitou "EmpresaXY" e a referência é "EmpresaXZ", então há uma substituição.
  - $D$ é o número de deleções. Por exemplo, se o usuário digitou "Empresa" e a referência é "EmpresaX", então há uma deleção.
  - $I$ é o número de inserções. Por exemplo, se o usuário digitou "Empresa XY" e a referência é "Empresa X", então há uma inserção.
  - $N$ é o número total de caracteres na referência. Por exemplo, se a referência é "Empresa X", então $N$ é 9 (contando espaços).

***
> ## Distância de Levenshtein Normalizada: 

Mede a diferença entre duas sequências de caracteres. É definida como o número mínimo de operações de edição (inserções, deleções ou substituições) necessárias para transformar uma palavra em outra. A normalização é feita dividindo a distância pelo comprimento da maior palavra entre as comparadas:
  $$D(A, B) = \frac{L(A, B)}{max(|A|, |B|)}$$
  onde $L(A, B)$ é a distância de Levenshtein entre as sequências $A$ e $B$, e $|A|$ e $|B|$ são os comprimentos das sequências.

  Por exemplo, se temos o texto "empresa X" e "empresa Y", a distância de Levenshtein seria 1 (substituindo "X" por "Y"). A normalização seria:
  $$D(\text{"empresa X"}, \text{"empresa Y"}) = \frac{1}{9}$$

***
> ## Similaridade de Jaccard: 

Mede a similaridade entre dois conjuntos. É definida como o tamanho da interseção dividido pelo tamanho da união dos conjuntos.
  $$J(A, B) = \frac{|A \cap B|}{|A \cup B|}$$

  No caso de textos, podemos considerar conjuntos de palavras ou caracteres. Por exemplo, se temos dois textos "empresa X" e "empresa Y", podemos considerar os conjuntos de palavras {empresa, X} e {empresa, Y}. A similaridade de Jaccard seria calculada como:
  $$J(\{empresa, X\}, \{empresa, Y\}) = \frac{|\{empresa\}|}{|\{empresa, X, Y\}|} = \frac{1}{3}$$

***
> ## Similaridade de Jaccard N Gram: 

É uma extensão da similaridade de Jaccard que considera n-gramas (sequências de n itens contíguos) em vez de palavras ou caracteres individuais. É útil para capturar similaridades em sequências mais longas, como frases ou sentenças. 
  A fórmula é semelhante à similaridade de Jaccard, mas aplicada a n-gramas:
  $$J(A, B) = \frac{|A_n \cap B_n|}{|A_n \cup B_n|}$$

No caso de textos, se temos o texto "empresa X", seus dois-gramas seriam {"em", "mp", "pr", "re", "sa", "a ", " X"}. Seus três-gramas seriam {"emp", "mpr", "pre", "res", "esa", "sa ", "a X"} e assim por diante. A similaridade de Jaccard N Gram seria calculada considerando esses n-gramas.

Ela é útil para capturar similaridades em casos de comparação de textos com erros de digitação, com pequenas inversoes/omissoes de caracteres, como ao comparar "emprEsa X" e "emprsa X", onde a ordem dos caracteres é preservada, mas algumas letras estão fora de lugar.

***

> ## TF-IDF (Term Frequency-Inverse Document Frequency):

É uma técnica de pontuação que mede a importância relativa de palavras em documentos. No contexto de matching entre input do usuário e registros de nome_fantasia/razao_social, o TF-IDF ajuda a identificar quais termos são mais distintivos para cada empresa, priorizando palavras raras e específicas sobre termos genéricos comuns.

A fórmula combina dois componentes:

**TF (Term Frequency):**
$$TF(t,d) = \frac{\text{número de ocorrências do termo t no nome de empresa d}}{\text{número total de termos no nome de empresa d}}$$

**IDF (Inverse Document Frequency):**
$$IDF(t,D) = \log\left(\frac{\text{número total de nomes distintos de empresas no corpus}}{\text{número de nomes de empresas que contêm o termo t}}\right)$$

**TF-IDF final:**
$$TF\text{-}IDF(t,d,D) = TF(t,d) \times IDF(t,D)$$

**Onde:**
- **t** = termo/palavra específica (ex: "Petrobras", "S.A.", "Banco")
- **d** = no caso do problema, a razao social de uma empresa concatenada com seu nome fantasia, separados por um espaco (ex: "Apple Inc. Apple")
- **D** = corpus completo com os nomes de todas as empresas definidas como o **d** acima.

**Vetor Final:**
Para cada nome de empresa **d**, o vetor TF-IDF tem dimensão igual ao número $n$ de termos únicos que formam todos os nomes de empresas no corpus $D$, onde cada posição representa o score TF-IDF de um termo:
$$\vec{v_d} = [TF\text{-}IDF(t_1,d,D), TF\text{-}IDF(t_2,d,D), ..., TF\text{-}IDF(t_n,d,D)]$$

O resultado desse processo é um **índice de vetores** em que cada um representa uma empresa específica. 

Termos raros em relacao a todos os nomes de empresas disponíveis com excecao daquela empresa a qual o termo se refere terao IDF grandes. Por exemplo, suponha que **D** é formado por nomes de empresas como "Petrobras S.A.", "Magazine Luiza S.A." e "Banco do Brasil S.A.". Nesse caso, a palavra "S.A." aparece em muitas empresas, fazendo que seu IDF seja baixo nos vetores $\vec{v_d}$ todos os nomes de empresas e, portanto, nao tendo relevancia para distinguir nomes de empresas distintas.

"Petrobras" aparece no nome de uma única empresa, portanto terá IDF alto. Quando o usuário digita "petrobras", o seu vetor TF-IDF é calculado para todos os nomes de empresas $d$ e o nome de empresa $d$ correspondente retornado pelo retriever é o aquele cujo vetor $\vec{v_d}$ é o mais similar ao vetor correspondente ao input do usuário.

In [ ]:
class TextMetrics:
    """
    A class containing various text comparison metrics with input validation.
    """

    @staticmethod
    def calculate_cer(reference: str, hypothesis: str) -> float:
        """Calculate Character Error Rate."""
        if not reference or not hypothesis:
            return 1.0  # Worst case: 100% error
        return cer(reference, hypothesis)

    @staticmethod
    def calculate_wer(reference: str, hypothesis: str) -> float:
        """Calculate Word Error Rate."""
        if not reference or not hypothesis:
            return 1.0  # Worst case: 100% error
        return wer(reference, hypothesis)

    @staticmethod
    def calculate_normalized_levenshtein(reference: str, hypothesis: str) -> float:
        """
        Calculate normalized Levenshtein distance (0-1).
        Returns:
            float: Normalized Levenshtein distance between 0 and 1
        """
        if not reference or not hypothesis:
            return 1.0  # Worst case: maximum distance
        max_len = max(len(reference), len(hypothesis))
        if max_len == 0:
            return 0.0
        
        distance = Levenshtein.distance(reference, hypothesis)
        return distance / max_len

    @staticmethod
    def _get_character_set(text: str) -> Set[str]:
        """
        Convert text to a set of characters.
        
        Args:
            text (str): Input text
            
        Returns:
            Set[str]: Set of characters from the input text
        """
        return set(text)

    @staticmethod
    def _get_word_set(text: str) -> Set[str]:
        """
        Convert text to a set of words.
        
        Args:
            text (str): Input text
            
        Returns:
            Set[str]: Set of words from the input text
        """
        return set(text.lower().split())

    @staticmethod 
    def ngram_jaccard_similarity(reference: str, hypothesis: str, n=2):
        """
        Calculate Jaccard similarity using character n-grams.
        This handles inversions and some misspellings well.
        
        Args:
            reference, hypothesis: Input strings
            n: N-gram size (2=bigrams, 3=trigrams, etc.)
        """
        if not reference or not hypothesis:
            return 0.0  # Worst case: no similarity

        def get_ngrams(text, n):
            """Generate n-grams from text with padding."""
            # Add padding to capture beginning/end patterns
            padded = '#' * (n-1) + text.lower() + '#' * (n-1)
            return set(padded[i:i+n] for i in range(len(padded) - n + 1))
        
        ngrams1 = get_ngrams(reference, n)
        ngrams2 = get_ngrams(hypothesis, n)
        
        intersection = len(ngrams1 & ngrams2)
        union = len(ngrams1 | ngrams2)
        
        return intersection / union if union > 0 else 1.0 if len(reference) == len(hypothesis) == 0 else 0.0

## 3.1 Implementação do Retriever

O retriever será implementado na classe `TextRetriever`.

O método `TextRetrieval.find_best_matches` recebe um `user_input` e um dataframe contendo apenas as entradas correspondentes ao estado/`uf` daquele input e retorna os `k` `razaosocial` e `nome_fantasia` mais similares.


In [ ]:
from typing import List, Tuple, Callable
from enum import Enum
import pandas as pd
from collections import Counter

class SimilarityMetric(Enum):
    CER = "cer"
    WER = "wer"
    LEVENSHTEIN = "levenshtein"
    NGRAM_JACCARD = "ngram_jaccard"
    TFIDF = "tfidf"


class TextRetrieval:
    """
    A class for text retrieval using similarity metrics.
    """

    _tfidf_cache = {}
    
    # The values of this dictionary are tuples in which the index 1 holds
    # a boolean that tells whether a metric is such that the higher 
    # its value the more similar the strings being compared are
    _METRIC_CONFIG = {
        SimilarityMetric.CER: (TextMetrics.calculate_cer, False),
        SimilarityMetric.WER: (TextMetrics.calculate_wer, False),
        SimilarityMetric.LEVENSHTEIN: (TextMetrics.calculate_normalized_levenshtein, False),
        SimilarityMetric.NGRAM_JACCARD: (TextMetrics.ngram_jaccard_similarity, True),
        SimilarityMetric.TFIDF: (None, True),  
    }

    @staticmethod
    def _get_tfidf_cache_key(df: pd.DataFrame) -> str:
        return str(hash(pd.util.hash_pandas_object(df[["razaosocial_cleaned", "nome_fantasia_cleaned"]], index=False).sum()))

    @classmethod
    def _get_tfidf_cache(cls, df: pd.DataFrame):
        key = cls._get_tfidf_cache_key(df)
        if key not in cls._tfidf_cache:
            combined = (
                df["razaosocial_cleaned"].fillna('') + ' ' + df["nome_fantasia_cleaned"].fillna('')
            )
            vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 4))
            tfidf_matrix = vectorizer.fit_transform(combined)
            cls._tfidf_cache[key] = (vectorizer, tfidf_matrix, combined)
        return cls._tfidf_cache[key]

    @staticmethod
    def _retrieve_topk_cnpjs_from_pairs(
        df: pd.DataFrame,
        razao_social_list: List[str],
        nome_fantasia_list: List[str],
        cnpj_col: str = "cnpj",
        not_cleaned_razao_col: str = "razaosocial",
        not_cleaned_fantasia_col: str = "nome_fantasia",
        top_k: int = 5
    ) -> List[str]:
        assert len(razao_social_list) == len(nome_fantasia_list)
        mask = pd.Series(False, index=df.index)
        for razao, fantasia in zip(razao_social_list, nome_fantasia_list):
            mask |= ((df[not_cleaned_razao_col] == razao) & (df[not_cleaned_fantasia_col] == fantasia))
        cnpjs = df[mask][cnpj_col].dropna().tolist()
        most_common = Counter(cnpjs).most_common(top_k)
        return [cnpj for cnpj, _ in most_common]

    @classmethod
    def find_best_matches(
        cls,
        user_input: str, 
        df: pd.DataFrame, 
        metric: SimilarityMetric, 
        top_k: int = 1
    ) -> Tuple[List[str], List[str], List[float], List[float], List[str]]:

        if metric == SimilarityMetric.TFIDF:
            return cls._find_matches_tfidf(user_input, df, top_k)

        if metric not in cls._METRIC_CONFIG:
            raise ValueError(f"Unsupported metric: {metric}")
        
        metric_func, reverse_sort = cls._METRIC_CONFIG[metric]

        return cls._find_matches_with_metric(
            user_input, df, metric_func, reverse_sort, top_k
        )

    @classmethod
    def _find_matches_tfidf(
        cls,
        user_input: str,
        df: pd.DataFrame,
        top_k: int,
        not_cleaned_razao_col: str = "razaosocial",
        not_cleaned_fantasia_col: str = "nome_fantasia",
        cnpj_col: str = "cnpj"
    ) -> Tuple[List[str], List[str], List[str]]:

        vectorizer, tfidf_matrix, combined_col = cls._get_tfidf_cache(df)
        query_vec = vectorizer.transform([user_input])
        similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
        top_indices = np.argsort(similarities)[::-1][:top_k]

        best_razao_matches = df.iloc[top_indices][not_cleaned_razao_col].tolist()
        best_nome_fantasia_matches = df.iloc[top_indices][not_cleaned_fantasia_col].tolist()
        best_cnpj_matches =  df.iloc[top_indices][cnpj_col].tolist()

        return (
            best_razao_matches,
            best_nome_fantasia_matches,
            best_cnpj_matches,
            )

    @classmethod
    def _find_matches_with_metric(
        cls,
        user_input: str,
        df: pd.DataFrame,
        metric_func: Callable[[str, str], float],
        reverse_sort: bool,
        top_k: int,
        razao_col: str = "razaosocial_cleaned",
        fantasia_col: str = "nome_fantasia_cleaned",
        not_cleaned_razao_col: str = "razaosocial",
        not_cleaned_fantasia_col: str = "nome_fantasia",
    ) -> Tuple[List[str], List[str], List[float], List[float], List[str]]:

        valid_mask = ~(df[razao_col].isna() & df[fantasia_col].isna())
        if not valid_mask.any():
            return [], [], [], [], []

        df_valid = df[valid_mask].copy()

        razao_scores = df_valid[razao_col].apply(
            lambda x: metric_func(x, user_input) if pd.notna(x) else None
        )
        nome_scores = df_valid[fantasia_col].apply(
            lambda x: metric_func(x, user_input) if pd.notna(x) else None
        )

        if reverse_sort:
            max_scores = np.maximum(
                razao_scores.fillna(-np.inf), 
                nome_scores.fillna(-np.inf)
            )
        else:
            max_scores = np.minimum(
                razao_scores.fillna(np.inf), 
                nome_scores.fillna(np.inf)
            )

        sorted_indices = np.argsort(max_scores)
        if reverse_sort:
            sorted_indices = sorted_indices[::-1]

        top_indices = sorted_indices[:top_k]
        best_razao_matches = df_valid.iloc[top_indices][not_cleaned_razao_col].tolist()
        best_nome_fantasia_matches = df_valid.iloc[top_indices][not_cleaned_fantasia_col].tolist()
        best_cnpj_matches = cls._retrieve_topk_cnpjs_from_pairs(
            df_valid,
            best_razao_matches,
            best_nome_fantasia_matches,
            top_k=top_k
        )

        return best_razao_matches, best_nome_fantasia_matches, best_cnpj_matches

## 3.2 Escolha da Melhor Métrica


Serao sorteadas `1000` linhas do conjunto de dados para calcular as métricas de similaridade e erro.

 Para cada linha, vamos calcular todas as métricas de similaridade mencionadas acima entre o `user_input_cleaned` e as colunas `razaosocial_cleaned` e `nome_fantasia_cleaned`. A métrica escolhida será aquela que apresentar o menor valor de erro.

In [ ]:
from tqdm import tqdm
import pandas as pd

def evaluate_matching(df_sample: pd.DataFrame,
                      df_cleaned: pd.DataFrame,
                      metric: SimilarityMetric,
                      top_k: int =5):
    results = []

    # Pre-group df_cleaned by UF for faster access
    df_by_uf = {
        uf: group.drop_duplicates(subset=['razaosocial_cleaned', 'nome_fantasia_cleaned','cnpj',])
        for uf, group in df_cleaned.groupby('uf')
    }

    indexes = []
    results_dict = {}
    for idx, row in tqdm(df_sample.iterrows(), total=len(df_sample), desc="Processing"):
        user_input = row['user_input_cleaned']
        razaosocial = row['razaosocial_cleaned']
        nome_fantasia = row['nome_fantasia_cleaned']
        not_cleaned_user_input = row['user_input']
        true_razaosocial = row['razaosocial']
        true_nome_fantasia = row['nome_fantasia']
        cnpj = row['cnpj']
        uf = row['uf']

        df_uf = df_by_uf.get(uf)
        if df_uf is None or df_uf.empty:
            continue  # Skip if no data for that UF

        # One fast match call
        top_k_razao, top_k_nome, top_k_cnpj = TextRetrieval.find_best_matches(
            user_input, df_uf, metric, top_k=top_k
        )

        top_1_razao = top_k_razao[0] if top_k_razao else None
        top_1_nome = top_k_nome[0] if top_k_nome else None
        top_1_cnpj = top_k_cnpj[0] if top_k_cnpj else None

        result_row = {
            "user_input": user_input,
            "user_input_not_cleaned": not_cleaned_user_input,
            "razaosocial_not_cleaned": true_razaosocial,
            "nome_fantasia_not_cleaned": true_nome_fantasia,
            "razaosocial": razaosocial,
            "nome_fantasia": nome_fantasia,
            "cnpj": cnpj,
            "uf": uf,
            
            "top_1_razaosocial_retrieved": top_1_razao,
            "top_1_nomefantasia_retrieved": top_1_nome,
            "top_1_cnpj_retrieved": top_1_cnpj,

            "top_1_razaosocial_pred": top_1_razao == true_razaosocial if top_1_razao else False,
            "top_1_nomefantasia_pred": top_1_nome == true_nome_fantasia if top_1_nome else False,
            "top_1_cnpj_pred": top_1_cnpj == cnpj if top_1_cnpj else False,

            "top_5_razaosocial_pred": true_razaosocial in top_k_razao,
            "top_5_nomefantasia_pred": true_nome_fantasia in top_k_nome,
            "top_5_cnpj_pred": cnpj in top_k_cnpj,


        }

        # Store results in a dictionary for easy access
        results_dict[idx] = result_row
        results_dict[idx]['top_k_razaosocial_pred'] = top_k_razao
        results_dict[idx]['top_k_nomefantasia_pred'] = top_k_nome
        results_dict[idx]['top_k_cnpj_pred'] = top_k_cnpj
        

        results.append(result_row)
        indexes.append(idx)

    results_df = pd.DataFrame(results, index=indexes)
    return results_df, results_dict


In [ ]:
metrics = [SimilarityMetric.CER, 
           SimilarityMetric.WER,
           SimilarityMetric.LEVENSHTEIN,
           SimilarityMetric.NGRAM_JACCARD,
           SimilarityMetric.TFIDF]

sample_size = 1000
top_k = 5
df_sample = df_cleaned.sample(n=sample_size, random_state=42)
accuracies = {metric.name: {} for metric in metrics}
all_results_df = pd.DataFrame()
for metric in metrics:
    results_df, results_dict = evaluate_matching(df_sample, df_cleaned, metric, top_k=top_k)
    results_df['metric'] = metric.name 
    all_results_df = pd.concat([all_results_df, results_df], ignore_index=True)

In [ ]:
accuracies = {}
for metric in metrics:
    metric_name = metric.name
    metric_df = all_results_df[all_results_df['metric'] == metric_name]
    accuracies[metric_name] = {
        'top_1_razaosocial': metric_df['top_1_razaosocial_pred'].mean(),
        'top_1_nomefantasia': metric_df['top_1_nomefantasia_pred'].mean(),
        'top_1_cnpj': metric_df['top_1_cnpj_pred'].mean(),
        'top_5_razaosocial': metric_df['top_5_razaosocial_pred'].mean(),
        'top_5_nomefantasia': metric_df['top_5_nomefantasia_pred'].mean(),
        'top_5_cnpj': metric_df['top_5_cnpj_pred'].mean()
    }

cases = ['top_1_razaosocial', 'top_1_nomefantasia', 'top_1_cnpj', 'top_5_razaosocial', 'top_5_nomefantasia', 'top_5_cnpj']
print("=" * 100)
for case in cases:
    sorted_metrics = sorted(accuracies.items(), key=lambda x: x[1][case], reverse=True)
    print(f"Rankings for {case}:")
    for rank, (metric_name, _) in enumerate(sorted_metrics, start=1):
        accuracy = accuracies[metric_name][case] * 100
        print(f"{rank}. {metric_name}: {accuracy:.2f}%")
    print("=" * 100)

Como observado acima, a métrica de TF-IDF com a similaridade do cosseno foi significativamente melhor em todos os casos. Assim, iremos utilizar essa métrica para calcular os resultados em todo o conjunto de dados além das 1000 amostras já utilizadas.

In [ ]:
metric = SimilarityMetric.TFIDF
results_df, results_dict = evaluate_matching(df_cleaned, df_cleaned, metric, top_k=top_k)
print("="* 100)
print(f"\nResultados da Avaliação para a métrica: {metric.name}"
      f"\nTamanho do DataFrame de Resultados: {results_df.shape}"
      f"\nNúmero de Linhas com Predições Corretas (Top 1 Razão Social): {results_df['top_1_razaosocial_pred'].sum()}"
      f"\nNúmero de Linhas com Predições Corretas (Top 1 Nome Fantasia): {results_df['top_1_nomefantasia_pred'].sum()}"
      f"\nNúmero de Linhas com Predições Corretas (Top 1 CNPJ): {results_df['top_1_cnpj_pred'].sum()}"
      f"\nNúmero de Linhas com Predições Corretas (Top 5 Razão Social): {results_df['top_5_razaosocial_pred'].sum()}"
      f"\nNúmero de Linhas com Predições Corretas (Top 5 Nome Fantasia): {results_df['top_5_nomefantasia_pred'].sum()}"
      f"\nNúmero de Linhas com Predições Corretas (Top 5 CNPJ): {results_df['top_5_cnpj_pred'].sum()}"
      f"\nAcuracias:\n"
      f"Top 1 Razão Social: {results_df['top_1_razaosocial_pred'].mean() * 100:.2f}%"
      f"\nTop 1 Nome Fantasia: {results_df['top_1_nomefantasia_pred'].mean() * 100:.2f}%"
      f"\nTop 1 CNPJ: {results_df['top_1_cnpj_pred'].mean() * 100:.2f}%"
      f"\nTop 5 Razão Social: {results_df['top_5_razaosocial_pred'].mean() * 100:.2f}%"
      f"\nTop 5 Nome Fantasia: {results_df['top_5_nomefantasia_pred'].mean() * 100:.2f}%"
      f"\nTop 5 CNPJ: {results_df['top_5_cnpj_pred'].mean() * 100:.2f}%")
print("="* 100)


In [ ]:
results_df.to_csv("tf-idf-results.csv")

## 3.3 Análise dos Erros de CNPJ

Como observado acima, o `TextRetriever` implementado com o `TF-IDF` e `Cosine Similarity` apresentou os melhores resultados. Por outro lado, mesmo o TF-IDF nao obteve um resultado satisfatório para o `CNPJ` retornado no top 1 ou top 5, embora os resultados para a `razaosocial` e `nome_fantasia` nao tenham sido ruins.

Vamos observar como os erros de CNPJ no top 5 se distribuem em relacao aos erros e acertos de `razaosocial` e `nome_fantasia`.

Uma hipótese para o poeque isso acontece é que dentro de um mesmo estado/`uf` um mesmo par `razaosocial` e `nome_fantasia` pode ter diferentes `CNPJ`s, o que torna a tarefa de recuperação de `CNPJ` mais complexa mesmo quando se obtem o par `razaosocial` e `nome_fantasia` corretos. Para verificar se isso acontece, observemos como os erros de CNPJ se distribuem quando há acertos e/ou erros para a `razaosocial` e `nome_fantasia`.


In [ ]:
erro_top5_razao_nome = results_df[
    (results_df['top_5_razaosocial_pred'] == False) & 
    (results_df['top_5_nomefantasia_pred'] == False) &
    (results_df['top_5_cnpj_pred'] == False)
]

erro_top5_razao_acerto_nome = results_df[
    (results_df['top_5_razaosocial_pred'] == False) & 
    (results_df['top_5_nomefantasia_pred'] == True) &
    (results_df['top_5_cnpj_pred'] == False)
]
erro_top5_acerto_razao_nome = results_df[
    (results_df['top_5_razaosocial_pred'] == True) & 
    (results_df['top_5_nomefantasia_pred'] == False) &
    (results_df['top_5_cnpj_pred'] == False)
]

acerto_top5_razao_nome = results_df[
    (results_df['top_5_razaosocial_pred'] == True) & 
    (results_df['top_5_nomefantasia_pred'] == True) &
    (results_df['top_5_cnpj_pred'] == False)
]

total_cnpj_errors = results_df[
    (results_df['top_5_cnpj_pred'] == False)
]
print("\nDistribuição dos Erros de CNPJ:")
print(f"\nErro no Top 5 Razão Social e Nome Fantasia: {len(erro_top5_razao_nome)/len(total_cnpj_errors) * 100:.2f}%")
print(f"Erro no Top 5 Razão Social e acerto no Top 5 Nome Fantasia: {len(erro_top5_razao_acerto_nome)/len(total_cnpj_errors) * 100:.2f}%")
print(f"Acerto no Top 5 Razão Social e erro no Top 5 Nome Fantasia: {len(erro_top5_acerto_razao_nome)/len(total_cnpj_errors) * 100:.2f}%")
print(f"Acerto no Top 5 Razão Social e Nome Fantasia: {len(acerto_top5_razao_nome)/len(total_cnpj_errors) * 100:.2f}%")


Como verificado acima a maioria dos erros de CNPJ (aproximadamente **87%**) ocorre quando  houve acerto na `razaosocial` e/ou `nome_fantasia`, sendo que **65%** dos erros de CNPJ no top 5 ocorre em casos em que houve acerto de ambos `razaosocial` e `nome_fantasia`, mas o CNPJ retornado não é o correto. 

Isso ocorre provavelmente porque no conjunto de dados, em um mesmo estado (`uf`), há mais de uma empresa com os mesmos `razaosocial` e `nome_fantasia`. Verifiquemos se isso é verdade.

In [ ]:
cnpj_counts_per_pair = df.groupby(['uf', 'razaosocial', 'nome_fantasia'])['cnpj'].nunique()

total_records_per_uf = df.groupby('uf').size()

multiple_cnpjs_summary = (
    cnpj_counts_per_pair[cnpj_counts_per_pair > 1]
    .reset_index()
    .groupby('uf')
    .agg({
        'cnpj': ['count', 'mean', 'max']
    })
)
multiple_cnpjs_summary.columns = ['pairs_with_multiple_cnpjs', 'avg_cnpjs_per_problematic_pair', 'max_cnpjs_per_pair']

multiple_cnpjs_summary['total_records'] = total_records_per_uf

records_in_multi_cnpj = (
    df[df.groupby(['uf', 'razaosocial', 'nome_fantasia'])['cnpj'].transform('nunique') > 1]
    .groupby('uf')
    .size()
)

multiple_cnpjs_summary['records_in_multi_cnpj'] = records_in_multi_cnpj.reindex(multiple_cnpjs_summary.index, fill_value=0)

multiple_cnpjs_summary['percentage_records_in_multi_cnpj'] = (
    multiple_cnpjs_summary['records_in_multi_cnpj'] / 
    multiple_cnpjs_summary['total_records'] * 100
).round(2)

multiple_cnpjs_summary = multiple_cnpjs_summary[[
    'pairs_with_multiple_cnpjs', 
    'percentage_records_in_multi_cnpj',
    'records_in_multi_cnpj',
    'total_records',
]].round(2).sort_values('pairs_with_multiple_cnpjs', ascending=False)

print("Summary of pairs with multiple CNPJs per UF:")
multiple_cnpjs_summary

A tabela acima revela que em todos os `uf` presentes no conjunto de dados, há um número significativo de empresas com o mesmo `razaosocial` e `nome_fantasia`, mas com `CNPJ`s diferentes. 

A tarefa de recuperação de `CNPJ` de uma empresa em determinado estado (`uf`) necessita, entao, de mais informacoes alem da `razaosocial` e `nome_fantasia` da empresa buscada.

# 4. Criacao de Retriever Utilizando Sentence Transformers

Vamos implementar um retriever utilizando embeddings de sentenças.

 Serao gerados embeddings para as colunas `razaosocial_cleaned` e `nome_fantasia_cleaned`. Tais embeddings serão utilizados para calcular a similaridade entre o `user_input_cleaned` e as colunas mencionadas. A métrica de similaridade utilizada será a similaridade cosseno, que é uma métrica comum para medir a similaridade entre vetores de alta dimensão e é a mesma utilizada anteriormente para calcular a similaridade entre os vetores TF-IDF.

In [ ]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from collections import Counter

class SemanticRetrieval:
    def __init__(self, df: pd.DataFrame, model_name: str = "all-MiniLM-L6-v2"):
        self.df = df.copy()
        self.model = SentenceTransformer(model_name)
        self.index = None
        self.embeddings = None
        self.id_map = None  # maps index -> df index
        self._prepare_index()

    def _prepare_index(self):
        self.df['razaosocial_cleaned'] = self.df['razaosocial'].apply(comprehensive_text_cleaning)
        self.df['nome_fantasia_cleaned'] = self.df['nome_fantasia'].apply(comprehensive_text_cleaning)
        self.df['combined'] = (
            self.df['razaosocial_cleaned'].fillna('') + ' ' +
            self.df['nome_fantasia_cleaned'].fillna('')
        )

        # Compute embeddings
        self.embeddings = self.model.encode(self.df['combined'].tolist(), show_progress_bar=True)
        self.embeddings = np.array(self.embeddings).astype('float32')

        # Create FAISS index
        dimension = self.embeddings.shape[1]
        self.index = faiss.IndexFlatL2(dimension)
        self.index.add(self.embeddings)

        # Store mapping from index -> dataframe row
        self.id_map = self.df.index.to_numpy()

    def search(self, user_input: str, top_k: int = 5, uf: str = None) -> dict:
        user_embedding = self.model.encode([user_input]).astype('float32')
        distances, indices = self.index.search(user_embedding, top_k)

        matched_rows = self.df.iloc[self.id_map[indices[0]]].copy()

        if uf:
            matched_rows = matched_rows[matched_rows['uf'] == uf]

        cnpjs = matched_rows['cnpj'].dropna().tolist()
        most_common_cnpjs = [cnpj for cnpj, _ in Counter(cnpjs).most_common(top_k)]

        return {
            'razaosocial': matched_rows['razaosocial'].tolist(),
            'nome_fantasia': matched_rows['nome_fantasia'].tolist(),
            'cnpjs': most_common_cnpjs,
            'distances': distances[0].tolist()
        }


In [ ]:
from tqdm import tqdm
import pandas as pd

def evaluate_matching_semantic(df_sample, df_cleaned, top_k=5):
    results = []
    indexes = []
    results_dict = {}

    retrievers_by_uf = {}
    for uf, group in df_cleaned.groupby('uf'):
        group = group.drop_duplicates(subset=['razaosocial_cleaned', 'nome_fantasia_cleaned', 'cnpj'])
        group = group.reset_index(drop=False) 
        retrievers_by_uf[uf] = SemanticRetrieval(group)

    for idx, row in tqdm(df_sample.iterrows(), total=len(df_sample), desc="Semantic Matching"):
        user_input = row['user_input_cleaned']
        razaosocial = row['razaosocial_cleaned']
        nome_fantasia = row['nome_fantasia_cleaned']
        not_cleaned_user_input = row['user_input']
        not_cleaned_razaosocial = row['razaosocial']
        not_cleaned_nome_fantasia = row['nome_fantasia']
        cnpj = row['cnpj']
        uf = row['uf']

        retriever = retrievers_by_uf.get(uf)
        if retriever is None:
            continue 

        result = retriever.search(user_input, top_k=top_k)

        top_k_razao = result['razaosocial']
        top_k_nome = result['nome_fantasia']
        top_k_cnpj = result['cnpjs']

        top_1_razao = top_k_razao[0] if top_k_razao else None
        top_1_nome = top_k_nome[0] if top_k_nome else None
        top_1_cnpj = top_k_cnpj[0] if top_k_cnpj else None

        result_row = {
            "user_input": user_input,
            "user_input_not_cleaned": not_cleaned_user_input,
            "razaosocial_not_cleaned": not_cleaned_razaosocial,
            "nome_fantasia_not_cleaned": not_cleaned_nome_fantasia,
            "razaosocial": razaosocial,
            "nome_fantasia": nome_fantasia,
            "cnpj": cnpj,
            "uf": uf,
            
            "top_1_razaosocial_retrieved": top_1_razao,
            "top_1_nomefantasia_retrieved": top_1_nome,
            "top_1_cnpj_retrieved": top_1_cnpj,

            "top_1_razaosocial_pred": top_1_razao == not_cleaned_razaosocial if top_1_razao else False,
            "top_1_nomefantasia_pred": top_1_nome == not_cleaned_nome_fantasia if top_1_nome else False,
            "top_1_cnpj_pred": top_1_cnpj == cnpj if top_1_cnpj else False,

            "top_5_razaosocial_pred": not_cleaned_razaosocial in top_k_razao,
            "top_5_nomefantasia_pred": not_cleaned_nome_fantasia in top_k_nome,
            "top_5_cnpj_pred": cnpj in top_k_cnpj,
        }

        results_dict[idx] = result_row
        results_dict[idx]['top_k_razaosocial_pred'] = top_k_razao
        results_dict[idx]['top_k_nomefantasia_pred'] = top_k_nome
        results_dict[idx]['top_k_cnpj_pred'] = top_k_cnpj

        results.append(result_row)
        indexes.append(idx)

    results_df = pd.DataFrame(results, index=indexes)
    return results_df, results_dict


In [ ]:
sample_size = 1000
top_k = 5
if sample_size:
    df_sample = df_cleaned.sample(n=sample_size, random_state=42)
else:
    df_sample = df_cleaned
results_df, results_dict = evaluate_matching_semantic(df_sample, df_cleaned, top_k=top_k)

In [ ]:
print("\nResultados da Avaliação:"
      f"\nTamanho do DataFrame de Resultados: {results_df.shape}"
      f"\nNúmero de Linhas com Predições Corretas (Top 1 Razão Social): {results_df['top_1_razaosocial_pred'].sum()}"
      f"\nNúmero de Linhas com Predições Corretas (Top 1 Nome Fantasia): {results_df['top_1_nomefantasia_pred'].sum()}"
      f"\nNúmero de Linhas com Predições Corretas (Top 1 CNPJ): {results_df['top_1_cnpj_pred'].sum()}"
      f"\nNúmero de Linhas com Predições Corretas (Top 5 Razão Social): {results_df['top_5_razaosocial_pred'].sum()}"
      f"\nNúmero de Linhas com Predições Corretas (Top 5 Nome Fantasia): {results_df['top_5_nomefantasia_pred'].sum()}"
      f"\nNúmero de Linhas com Predições Corretas (Top 5 CNPJ): {results_df['top_5_cnpj_pred'].sum()}"
      f"\nAcuracias:\n"
      f"Top 1 Razão Social: {results_df['top_1_razaosocial_pred'].mean() * 100:.2f}%"
      f"\nTop 1 Nome Fantasia: {results_df['top_1_nomefantasia_pred'].mean() * 100:.2f}%"
      f"\nTop 1 CNPJ: {results_df['top_1_cnpj_pred'].mean() * 100:.2f}%"
      f"\nTop 5 Razão Social: {results_df['top_5_razaosocial_pred'].mean() * 100:.2f}%"
      f"\nTop 5 Nome Fantasia: {results_df['top_5_nomefantasia_pred'].mean() * 100:.2f}%"
      f"\nTop 5 CNPJ: {results_df['top_5_cnpj_pred'].mean() * 100:.2f}%")

Para um subconjunto de 1000 amostras do conjunto de dados, um modelo de sentence transformer nao tunado nao obteve um resultado satisfatório quando comparado ao retriever com TF-IDF. Portanto, por ora, considera-se a melhor solucao como sendo o retriever que utiliza o TF-IDF e a similaridade do cosseno.